# Step 7 Automation of Back Propagation


## 7.1 Premise

If we have to write the code for bp everytime there is a new calculation, it would be bothering us. Therefore, the automation of the process is inevitable.  

**Define-by-Run** would be explained in the further steps. 



**Relationship** between function and variable
1. function -> variable : exists as an **input** and **output**
2. variable -> function : can be called parents or **creator**

In [1]:
class Variable:
    def __init__(self, data):
        self.data=data
        self.grad=None
        self.creator=None
    
    def set_creator(self, func):
        self.creator=func
        
class Function:
    
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        output.set_creator(self) #set creator for the output variable
        
        self.input=input #remember the input variable
        self.output=output #also save the output
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, x):
        raise NotImplementedError()

In [5]:
import numpy as np

class Square(Function):
    def forward(self, x):
        y=x**2
        return y
    def backward(self, gy):
        x=self.input.data
        gx=gy*2*x
        return gx

class Exp(Function):
    def forward(self, x):
        y=np.exp(x)
        return y
    def backward(self, gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx

A= Square()
B= Exp()
C= Square()

x=Variable(np.array(0.5))

a=A(x)
b=B(a)
y=C(b)

assert y.creator==C
assert y.creator.input==b
assert y.creator.input.creator==B
assert y.creator.input.creator.input==a
assert y.creator.input.creator.input.creator==A
assert y.creator.input.creator.input.creator.input==x

## 7.3 Adding backward method

In [9]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data=data
        self.grad=None
        self.creator=None
    
    def set_creator(self, func):
        self.creator=func
    
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad=f.backward(self.grad)
            x.backward()

        
class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        output.set_creator(self) #set creator for the output variable   
        self.input=input #remember the input variable
        self.output=output #also save the output
        return output

class Square(Function):
    def forward(self, x):
        y=x**2
        return y
    
    def backward(self, gy):
        x=self.input.data
        gx=gy*2*x
        return gx
        
class Exp(Function):
    def forward(self, x):
        y=np.exp(x)
        return y
   
    def backward(self, gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx





In [10]:
A= Square()
B= Exp()
C= Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)

y.grad=np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
